In [24]:
from netmiko import ConnectHandler
from getpass import getpass
from jumpssh import SSHSession

ESW1 = {
    'device_type': 'cisco_ios',
    'ip': '10.145.61.10',
    'username': 'adm.davies',
    'password': '!"£Lepsodizle0!£"',
    'global_delay_factor': 4
}

In [25]:
jh_session = SSHSession('10.145.61.10','adm.davies',password='!"£Lepsodizle0!£"').open()
remote_connect = jh_session(**ESW1)
output = remote_connect.send_command("show router interface")
print(output)

ConnectionError: Unable to connect to '10.145.61.10:22' with user 'adm.davies': [Errno None] Unable to connect to port 22 on 10.145.61.10